In [1]:
import numpy as np
import pandas as pd
import nibabel as nib

Load a pre-made 4D image containing binary lesion masks from multiple patients.

In [2]:
mask_stack_path = '/home/despoB/lesion/anat_preproc/lesion_mask_mni_stack.nii.gz'

In [3]:
mask_stack_img = nib.load(mask_stack_path)

In [4]:
mask_stack_data = mask_stack_img.get_data()

In [5]:
mask_stack_data.shape

(91, 109, 91, 64)

Get the inverse affine transform for the image, so we can go from image-space coordinates to voxel-data coordiantes.

In [6]:
mask_stack_img.affine

array([[  -2.,   -0.,    0.,   90.],
       [  -0.,    2.,   -0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [7]:
inverse_affine = np.linalg.inv(mask_stack_img.affine)

Confirm that our coordinate translation works by checking against a known relationship (from FSLView).

MNI coordinate (28, 36, 2) = Voxel coordinate (31, 81, 37)

In [8]:
nib.affines.apply_affine(inverse_affine, [28, 36, 2])

array([ 31.,  81.,  37.])

To be double sure, test our transformed coordinates on real data. 

The patient 191 has a lesioned voxel at MNI coordinate (-36, 2, -2), but moving more than 2mm (1 voxel) in any direction puts you in empty space.

In [9]:
mask_path_191 = '/home/despoB/lesion/anat_preproc/191/191_mask_mni.nii.gz'

In [10]:
mask_img_191 = nib.load(mask_path_191)

In [11]:
mask_data_191 = mask_img_191.get_data()

There should be damage here..

In [12]:
nib.affines.apply_affine(inverse_affine, [-36, 2, -2])

array([ 63.,  64.,  35.])

In [13]:
mask_data_191[63, 64, 35]

1.0

But not in any of these places...

In [14]:
nib.affines.apply_affine(inverse_affine, [-40, 2, -2])

array([ 65.,  64.,  35.])

In [15]:
mask_data_191[65, 64, 35]

0.0

In [16]:
nib.affines.apply_affine(inverse_affine, [-36, -2, -2])

array([ 63.,  62.,  35.])

In [17]:
mask_data_191[63, 62, 35]

0.0

In [18]:
nib.affines.apply_affine(inverse_affine, [-36, 2, 2])

array([ 63.,  64.,  37.])

In [19]:
mask_data_191[63, 64, 37]

0.0

Looks like our coordinate conversion is working. These were lazy checks, but lazy is better than nothing.

Next, we need to map 3D voxel-data coordinates to 1D (flattened) coordinates.

First, create a small 3D array to test things on.

In [20]:
test_data = np.random.rand(3, 3, 3)

In [21]:
test_data.shape

(3, 3, 3)

In [22]:
test_data

array([[[ 0.16082858,  0.26996851,  0.06690347],
        [ 0.9089486 ,  0.24818446,  0.59843808],
        [ 0.12104779,  0.1486682 ,  0.59715994]],

       [[ 0.14908149,  0.25447388,  0.48833766],
        [ 0.41205626,  0.84955758,  0.40773006],
        [ 0.36975411,  0.3750414 ,  0.13871361]],

       [[ 0.98978035,  0.59917477,  0.91071247],
        [ 0.86275655,  0.78731706,  0.80127353],
        [ 0.13336423,  0.56304288,  0.57189325]]])

Pull an arbitrary item using 3D indexing.

In [23]:
test_data[0,2,1]

0.14866819987895685

The `ravel_multi_index` function translates a multi-dimensional index into the equivalent 1D index of a raveled array.

In [24]:
np.ravel_multi_index([0,2,1], (3,3,3))

7

In [25]:
test_data.ravel()[7]

0.14866819987895685

Now let's check this works on our image data.

In [26]:
mask_data_191[63, 64, 35]

1.0

In [27]:
np.ravel_multi_index([63, 64, 35], mask_data_191.shape)

630756

In [28]:
mask_data_191.ravel()[630756]

1.0

Now that we've got indexing working using NumPy arrays, let's translate things into Pandas so we can have labeled rows.

Construct a test DF with image vectors from 10 patients. We do it iteratively, adding one patient at a time. 

We'll also do some code profiling to see how much of a hit the system will take when doing these operations.

In [29]:
%load_ext memory_profiler

In [30]:
%%memit
for i in range(10):
    pdata = mask_stack_data[...,i].ravel()
    try:
        mask_data_df['10'+str(i)] = pdata
    except:
        mask_data_df = pd.DataFrame({'10'+str(i):pdata})

peak memory: 621.00 MiB, increment: 75.95 MiB


In [31]:
mask_data_df.head()

,100,101,102,103,104,105,106,107,108,109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now that we have the DF, let's search it.

In [32]:
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

['102', '103', '104', '105', '106']

In [33]:
%%timeit
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

1000 loops, best of 3: 1.02 ms per loop


In [34]:
%memit
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

peak memory: 722.08 MiB, increment: 0.00 MiB


['102', '103', '104', '105', '106']

Sure enough, visual inspection confirms that those patients have lesions at that coordinate.

Let's see if it is much slower when we tweak things so the outputs a list of patients directly.

In [35]:
%%timeit
mask_data_df.T[mask_data_df.T[630756] == 1].index

10 loops, best of 3: 31.4 ms per loop


Okay, so it looks like we definitely don't want to just transpose the DF on the fly. What if we do that beforehand.

In [36]:
mask_data_t = mask_data_df.T

In [37]:
%%timeit
mask_data_t[mask_data_t[630756] == 1].index

100 loops, best of 3: 15.7 ms per loop


Hmm, okay. It seems like the slowdown comes from indexing over columns instead of rows. Back to our original plan.

We want our mask database to be persistent, but fast to read and write, so we're going to use PyTables to store it as HDF5.

First, create an HDFstore object and add our DF to it.

In [38]:
store = pd.HDFStore('mask_data.h5')

In [39]:
store.put('df', mask_data_df, format='t')

In [40]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mask_data.h5
/df            frame_table  (typ->appendable,nrows->902629,ncols->10,indexers->[index])

In [41]:
store.df.head()

,100,101,102,103,104,105,106,107,108,109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can close the store to free up memory when we're not using the mask data.

In [42]:
store.close()
del store

When working with data stored as HD5, we can select only the rows/columns we want to be read into memory.

This makes searching really fast.

In [43]:
flat_index = 630756

In [44]:
%%timeit
restore = pd.HDFStore('mask_data.h5')
vox_row = restore.select('df', start=flat_index, stop=flat_index+1)
res = vox_row.T.iloc[:,0] == 1
patients = list(res[res == True].index)
restore.close()

100 loops, best of 3: 9.97 ms per loop


Much faster than reading the whole DF into memory.

In [45]:
%%timeit
store = pd.HDFStore('mask_data.h5')
test_results = store.df.loc[630756,:] == 1
patients = list(test_results[test_results == True].index)
store.close()

10 loops, best of 3: 112 ms per loop


We can now quickly seach existing patients, but how about adding mask data for a new patient?

In [46]:
store = pd.HDFStore('mask_data.h5')

In [47]:
data_df = store.get('df') # We have to get/modify a copy, because you can't add rows to an HD5 table.

In [48]:
data_df['191'] = mask_data_191.ravel()

In [49]:
store.put('df', data_df) # Overwrite the existing DF with the new DF.

In [50]:
store.close()

Double-check that `store` now contains the new patient data.

In [54]:
with pd.HDFStore('mask_data.h5') as store:
    print(store.df.columns)

Index(['100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '191'],
      dtype='object')


It's hacky, slow, and and will eat up a lot of memory as the DF grows, but it works for now. 

Now that we've got things working, we'll put everything into functions.

ALSO NEED FUNCTION TO BUILD/REBUILD MASK SUM IMAGE

In [56]:
import os 
def add_patient_mask(patient_number, mask_path, store_path):
    """
    Add a new patient/mask to the damage database.
    
    Parameters
    ----------
    patient_number : str
    mask_path : str
        Path the patient's lesion mask. Must be in MNI space,
        with 2mm voxels, and have RPI orientation.
    datastore : str
        Path to a Pandas HDF5store (e.g. file.h5)
    """
    # Check inputs
    assert isinstance(patient_number, str)
    assert os.path.exists(mask_path)
    
    # Load the mask image.
    mask_data = nib.load(mask_path).get_data()
    
    # Open the HD5 store, add a column for the new patient, then close the store.
    store = pd.HDFStore(store_path)
    data_df = store.get('df')
    data_df[patient_number] = mask_data.ravel()
    store.put('df', data_df)
    print(store.df.columns)
    store.flush()
    store.close()

In [58]:
%%time
add_patient_mask('192', '/home/despoB/lesion/anat_preproc/191/191_mask_mni.nii.gz','mask_data.h5')

Index(['100', '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '191', '192'],
      dtype='object')
CPU times: user 0 ns, sys: 1.56 s, total: 1.56 s
Wall time: 3.32 s


It's slow, but acceptable. Adding new patients won't happen all that often. May have to re-factor when the DF gets bigger.

In the future, we can try refactoring things to utilize `store.select` while storing each mask as a separate node in the HDFStore.